## 10. Point set and Mask Transformation

Transformix can be used to transform point sets and mask images aswell. Masks can be seen as images so the registration of masks is done in similar fashion as the registration of a moving image.
Point sets however need not be transformed with the backwards mapping protocol, but can instead be transformed with regular forward transformation (fixed -> moving)

Transforming point sets can be used to get the regions of interest (ROI) in the moving image, based on ROI of the fixed image.

### Elastix

In [1]:
# First two import are currently necessary to run ITKElastix on MacOs
from itk import itkElastixRegistrationMethodPython
from itk import itkTransformixFilterPython
import itk

In [2]:
# Import Images
fixed_image = itk.imread('data/CT_3D_lung_fixed.mha', itk.F)
moving_image = itk.imread('data/CT_3D_lung_moving.mha', itk.F)

# Import Default Parameter Map
parameter_object = itk.ParameterObject.New()
parameter_map_rigid = parameter_object.GetDefaultParameterMap('rigid')
parameter_object.AddParameterMap(parameter_map_rigid)

Registration with the registration function...

In [3]:
# Call registration function
result_image, result_transform_parameters = itk.elastix_registration_method(
    fixed_image, moving_image,
    parameter_object=parameter_object)

### Point Set Transformation

Transformation can either be done in one line with the transformix function...

In [4]:
# --- Error: procedural interface of transformix filter not working in 3D yet ---
# result_point_set = itk.transformix_filter(
#     moving_image=moving_image,
#     fixed_point_set_file_name='data/CT_3D_lung_fixed_point_set.txt',
#     transform_parameter_object=result_transform_parameters,
#     output_directory = '/exampleoutput')

.. or by initiating an transformix image filter object.

In [7]:
# Load Transformix Object
# Explicit call to 3D Transformix method is currently necessary 
# for 3D (point set) transformations, this might change in future versions.
transformix_object = itk.itkTransformixFilterPython.itkTransformixFilterIF3.New()
transformix_object.SetFixedPointSetFileName('data/CT_3D_lung_fixed_point_set.txt')
transformix_object.SetTransformParameterObject(result_transform_parameters)
transformix_object.SetLogToConsole(True)
transformix_object.SetOutputDirectory('exampleoutput/')

# Update object (required)
transformix_object.UpdateLargestPossibleRegion()

# Results of Transformation
# -- Bug? -- Output is saved as .txt file in outputdirectory.
# The .GetOutput() function outputs an empty image.
output_transformix = transformix_object.GetOutput()

### Mask Transformation

In [8]:
# Import Mask
moving_mask = itk.imread('data/CT_3D_lung_moving_mask.mha', itk.F)

Transformation of a mask is similar to the transformation of an image and can either be done in one line with the transformix function...

In [9]:
# --- Error: procedural interface of transformix filter not working in 3D yet ---
# result_mask = itk.transformix_filter(
#     moving_image=moving_mask,
#     transform_parameter_object=result_transform_parameters)

.. or by initiating an transformix image filter object.

In [10]:
# Load Transformix Object 
# Explicit call to 3D Transformix method is currently necessary 
# for 3D (point set) transformations, this might change in future versions.
transformix_object = itk.itkTransformixFilterPython.itkTransformixFilterIF3.New()
transformix_object.SetMovingImage(moving_mask)
transformix_object.SetTransformParameterObject(result_transform_parameters)

# Update object (required)
transformix_object.UpdateLargestPossibleRegion()

# Results of Transformation
result_moving_mask = transformix_object.GetOutput()